<a href="https://colab.research.google.com/github/nguyenngoctruong2k1/NCKH_IDS/blob/main/IDS/KDDCup99/GRU/lskeras3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import thư viện

In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

# Load data

In [2]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NCKH_IDS/kdd/data.csv', header=None)

Mounted at /content/drive


In [3]:
# protocol_type = {}
# service = {}
# flag = {}
# for i in range(len(data)):
#   if data.iloc[i,1] not in protocol_type:
#     protocol_type.update({data.iloc[i,1]:len(protocol_type)})
#   if data.iloc[i,2] not in service:
#     service.update({data.iloc[i,2]:len(service)})
#   if data.iloc[i,3] not in flag:
#     flag.update({data.iloc[i,3]:len(flag)})
protocol_type = {'icmp': 2, 'tcp': 0, 'udp': 1}
flag = {'OTH': 9,
        'REJ': 2,
        'RSTO': 6,
        'RSTOS0': 8,
        'RSTR': 7,
        'S0': 4,
        'S1': 1,
        'S2': 3,
        'S3': 5,
        'SF': 0,
        'SH': 10}
service = {
        'IRC': 39,
        'X11': 32,
        'Z39_50': 40,
        'aol': 65,
        'auth': 3,
        'bgp': 46,
        'courier': 44,
        'csnet_ns': 47,
        'ctf': 27,
        'daytime': 16,
        'discard': 31,
        'domain': 22,
        'domain_u': 2,
        'echo': 18,
        'eco_i': 6,
        'ecr_i': 9,
        'efs': 54,
        'exec': 45,
        'finger': 4,
        'ftp': 7,
        'ftp_data': 14,
        'gopher': 24,
        'harvest': 64,
        'hostnames': 55,
        'http': 0,
        'http_2784': 63,
        'http_443': 48,
        'http_8001': 67,
        'imap4': 35,
        'iso_tsap': 59,
        'klogin': 49,
        'kshell': 61,
        'ldap': 42,
        'link': 29,
        'login': 34,
        'mtp': 23,
        'name': 20,
        'netbios_dgm': 41,
        'netbios_ns': 60,
        'netbios_ssn': 51,
        'netstat': 15,
        'nnsp': 53,
        'nntp': 36,
        'ntp_u': 8,
        'other': 10,
        'pm_dump': 38,
        'pop_2': 52,
        'pop_3': 13,
        'printer': 50,
        'private': 12,
        'red_i': 69,
        'remote_job': 25,
        'rje': 26,
        'shell': 33,
        'smtp': 1,
        'sql_net': 57,
        'ssh': 17,
        'sunrpc': 43,
        'supdup': 28,
        'systat': 30,
        'telnet': 5,
        'tftp_u': 66,
        'tim_i': 68,
        'time': 19,
        'urh_i': 62,
        'urp_i': 11,
        'uucp': 37,
        'uucp_path': 56,
        'vmnet': 58,
        'whois': 21
}
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,0,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.00,1,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,2,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.00,3,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.00,4,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,tcp,http,SF,212,2288,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,32,0.0,0.0,0.0,0.0,1.0,0.0,0.16,3,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,219,236,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,33,0.0,0.0,0.0,0.0,1.0,0.0,0.15,4,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,218,3610,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,34,0.0,0.0,0.0,0.0,1.0,0.0,0.15,5,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1234,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,35,0.0,0.0,0.0,0.0,1.0,0.0,0.14,6,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.


In [4]:
def F_label(data):
  if data == 'normal.':
    return 0
  return 1
def SoHoaDuLieu(traindata):
  traindata[1] = traindata[1].map(protocol_type)
  traindata[3] = traindata[3].map(flag)
  traindata[41] = traindata[41].map(F_label)
  traindata[2] = traindata[2].map(service)
  return traindata
data = SoHoaDuLieu(data)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,0,0,0,0,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,0,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0
1,0,0,0,0,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.00,1,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,0
2,0,0,0,0,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,2,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,0
3,0,0,0,0,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.00,3,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,0
4,0,0,0,0,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.00,4,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,0,0,0,212,2288,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,32,0.0,0.0,0.0,0.0,1.0,0.0,0.16,3,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,0
4898427,0,0,0,0,219,236,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,33,0.0,0.0,0.0,0.0,1.0,0.0,0.15,4,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,0
4898428,0,0,0,0,218,3610,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,34,0.0,0.0,0.0,0.0,1.0,0.0,0.15,5,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,0
4898429,0,0,0,0,219,1234,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,35,0.0,0.0,0.0,0.0,1.0,0.0,0.14,6,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,0


In [5]:
Y = data[41]
X = data.drop(41, axis=1)
scaler = Normalizer().fit(X)
X = scaler.transform(X)

# Split test and train data 
trainX, testT, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [6]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))

print(X_train.shape)

(3428901, 1, 41)


In [7]:
y_train.shape

(3428901,)

In [8]:
X_train.shape

(3428901, 1, 41)

# Dựng model

In [10]:
batch_size = 32

# 1. define the network
model = Sequential()
model.add(GRU(32,input_dim=41, return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(GRU(32, return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(GRU(32, return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(GRU(32, return_sequences=False))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))

print(model.get_config())

{'name': 'sequential_1', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, None, 41), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'gru_4_input'}}, {'class_name': 'GRU', 'config': {'name': 'gru_4', 'trainable': True, 'batch_input_shape': (None, None, 41), 'dtype': 'float32', 'return_sequences': True, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 32, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.0, 'recurrent_dro

# Train model

In [11]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="/content/drive/MyDrive/Colab Notebooks/NCKH_IDS/kdd/kddresults/lstm4layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('/content/drive/MyDrive/Colab Notebooks/NCKH_IDS/kdd/kddresults/training_set_iranalysis4.csv',separator=',', append=False)
model.fit(X_train, y_train, batch_size=batch_size, epochs=20, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])

Epoch 1/20
107154/107154 [==============================] - 714s 7ms/step - loss: 0.0135 - accuracy: 0.9960 - val_loss: 0.0042 - val_accuracy: 0.9988
Epoch 2/20
107154/107154 [==============================] - 720s 7ms/step - loss: 0.0043 - accuracy: 0.9990 - val_loss: 0.0031 - val_accuracy: 0.9993
Epoch 3/20
107154/107154 [==============================] - 733s 7ms/step - loss: 0.0035 - accuracy: 0.9992 - val_loss: 0.0026 - val_accuracy: 0.9994
Epoch 4/20
107154/107154 [==============================] - 731s 7ms/step - loss: 0.0030 - accuracy: 0.9993 - val_loss: 0.0023 - val_accuracy: 0.9994
Epoch 5/20
107154/107154 [==============================] - 732s 7ms/step - loss: 0.0028 - accuracy: 0.9994 - val_loss: 0.0021 - val_accuracy: 0.9995
Epoch 6/20
107154/107154 [==============================] - 746s 7ms/step - loss: 0.0025 - accuracy: 0.9994 - val_loss: 0.0019 - val_accuracy: 0.9996
Epoch 7/20
107154/107154 [==============================] - 738s 7ms/step - loss: 0.0023 - accuracy:

In [12]:
model.save("/content/drive/MyDrive/Colab Notebooks/NCKH_IDS/kdd/kddresults/lstm4layer/fullmodel/lstm4layer_model.hdf5")

# Report

In [13]:
loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))

45923/45923 [==============================] - 87s 2ms/step - loss: 0.0011 - accuracy: 0.9998

Loss: 0.00, Accuracy: 99.98%


In [14]:
y_pred = model.predict_classes(X_test)
np.savetxt('/content/drive/MyDrive/Colab Notebooks/NCKH_IDS/kdd/kddresults/lstm4layer/lstm4predicted.txt', y_pred, fmt='%01d')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [15]:
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)

In [16]:
accuracy_score(y_test, y_pred)

0.9998223921934224

In [17]:
precision_score(y_test, y_pred, average="binary")

0.9999447946914576

In [18]:
f1_score(y_test, y_pred, average="binary")

0.9998891708921149

In [19]:
recall_score(y_test, y_pred , average="binary")

0.9998335532807837